In [1]:
import tensorflow as tf 
import numpy as np
import os 
import time 

path_to_file = ('recipes.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8') 
print(text[:250])


No-Bake Nut Cookies
1 c. firmly packed brown sugar
 1/2 c. evaporated milk
 1/2 tsp. vanilla
 1/2 c. broken nuts (pecans)
 2 Tbsp. butter or margarine
 3 1/2 c. bite size shredded rice biscuitsIn a heavy 2-quart saucepan, mix brown sugar, nuts, evapo


In [2]:
print(f'Length of text: {len(text)} characters')
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')


Length of text: 2390691 characters
86 unique characters


In [3]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')


In [4]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

ids = ids_from_chars(chars)

print(ids)

<tf.RaggedTensor [[60, 61, 62, 63, 64, 65, 66], [83, 84, 85]]>


In [5]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
    
chars = chars_from_ids(ids)
    
print(chars)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


In [6]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

print(text_from_ids(ids))

tf.Tensor([b'abcdefg' b'xyz'], shape=(2,), dtype=string)


In [7]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8')) 
print(all_ids) 


tf.Tensor([45 74 15 ... 16  2  2], shape=(2390691,), dtype=int64)


In [8]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))


N
o
-
B
a
k
e
 
N
u


In [9]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1):
    print(chars_from_ids(seq))

for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())


tf.Tensor(
[b'N' b'o' b'-' b'B' b'a' b'k' b'e' b' ' b'N' b'u' b't' b' ' b'C' b'o'
 b'o' b'k' b'i' b'e' b's' b'\n' b'1' b' ' b'c' b'.' b' ' b'f' b'i' b'r'
 b'm' b'l' b'y' b' ' b'p' b'a' b'c' b'k' b'e' b'd' b' ' b'b' b'r' b'o'
 b'w' b'n' b' ' b's' b'u' b'g' b'a' b'r' b'\n' b' ' b'1' b'/' b'2' b' '
 b'c' b'.' b' ' b'e' b'v' b'a' b'p' b'o' b'r' b'a' b't' b'e' b'd' b' '
 b'm' b'i' b'l' b'k' b'\n' b' ' b'1' b'/' b'2' b' ' b't' b's' b'p' b'.'
 b' ' b'v' b'a' b'n' b'i' b'l' b'l' b'a' b'\n' b' ' b'1' b'/' b'2' b' '
 b'c' b'.' b' '], shape=(101,), dtype=string)
b'No-Bake Nut Cookies\n1 c. firmly packed brown sugar\n 1/2 c. evaporated milk\n 1/2 tsp. vanilla\n 1/2 c. '
b'broken nuts (pecans)\n 2 Tbsp. butter or margarine\n 3 1/2 c. bite size shredded rice biscuitsIn a heav'
b'y 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.\n Stir over medium'
b' heat until mixture bubbles all over top.\n Boil and stir 5 minutes more. Take off heat.\n Stir in vani'
b'lla and cer

In [10]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

print(split_input_target(list("i love id tech"))) 


(['i', ' ', 'l', 'o', 'v', 'e', ' ', 'i', 'd', ' ', 't', 'e', 'c'], [' ', 'l', 'o', 'v', 'e', ' ', 'i', 'd', ' ', 't', 'e', 'c', 'h'])


In [11]:
dataset = sequences.map(split_input_target) 

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


Input : b'No-Bake Nut Cookies\n1 c. firmly packed brown sugar\n 1/2 c. evaporated milk\n 1/2 tsp. vanilla\n 1/2 c.'
Target: b'o-Bake Nut Cookies\n1 c. firmly packed brown sugar\n 1/2 c. evaporated milk\n 1/2 tsp. vanilla\n 1/2 c. '


In [12]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = (dataset .shuffle(BUFFER_SIZE) .batch(BATCH_SIZE, drop_remainder=True) .prefetch(tf.data.experimental.AUTOTUNE))
print(dataset)


<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [13]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024


In [14]:
class MyModel(tf.keras.Model):
 def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
 def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)
    if return_state:
      return x, states
    else:
      return x

model = MyModel(    
    vocab_size=len(ids_from_chars.get_vocabulary()),    
    embedding_dim=embedding_dim,    
    rnn_units=rnn_units)


In [15]:
for input_example_batch, target_example_batch in dataset.take(1): 
    example_batch_predictions = model(input_example_batch) 
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") 
model.summary()


(64, 100, 87) # (batch_size, sequence_length, vocab_size)
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  22272     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  89175     
                                                                 
Total params: 4,049,751
Trainable params: 4,049,751
Non-trainable params: 0
_________________________________________________________________


In [16]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
print (sampled_indices)
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())


[85 17 81 49  7 75 60 82 86 83 35 60 52 20 19 38 55 61 85 79 14 26 23 32
 21 47 36 79 15 74 71 85 68 19 40  5 52 61 68 64 71 21 15 42 22 49 27 15
 83 60 71  8 52 21 86 60  8 16 51 69 72 68 20 42 83 16 39 24 85 13 51 12
 41 82 85 83 34 35  6 68  5 53 21 59 26 49 81 67  7  4 76 14  0 42 33  2
 46 70 64  2]
Input:
 b'se Fruit Cake\n3 eggs\n 1 c. milk\n 3 c. flour\n 2 c. sugar\n 1 c. butter\n 1 tsp. allspice\n 1 tsp. cinnam'
Next Char Predictions:
 b'z/vR%paw~xDaU21GXbzt,85A3PEt-olzi1I"Ubiel3-K4R9-xal&U3~a&.Tjmi2Kx.H6z+T*JwzxCD#i"V3`8Rvh%!q,[UNK]KB\nOke\n'


In [17]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss: ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()


Prediction shape:  (64, 100, 87)  # (batch_size, sequence_length, vocab_size)
Mean loss:  tf.Tensor(4.467333, shape=(), dtype=float32)


87.12404

In [18]:
model.compile(optimizer='adam', loss=loss)
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_prefix, save_weights_only=True)
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])






Epoch 1/20
369/369 [==============================] - 451s 1s/step - loss: 2.0611
Epoch 2/20
369/369 [==============================] - 572s 2s/step - loss: 1.0306
Epoch 3/20
369/369 [==============================] - 541s 1s/step - loss: 0.8679
Epoch 4/20
369/369 [==============================] - 469s 1s/step - loss: 0.8006
Epoch 5/20
369/369 [==============================] - 440s 1s/step - loss: 0.7564
Epoch 6/20
369/369 [==============================] - 458s 1s/step - loss: 0.7226
Epoch 7/20
369/369 [==============================] - 429s 1s/step - loss: 0.6939
Epoch 8/20
369/369 [==============================] - 441s 1s/step - loss: 0.6681
Epoch 9/20
369/369 [==============================] - 465s 1s/step - loss: 0.6443
Epoch 10/20
369/369 [==============================] - 454s 1s/step - loss: 0.6220
Epoch 11/20
369/369 [==============================] - 446s 1s/step - loss: 0.6013
Epoch 12/20
369/369 [==============================] - 437s 1s/step - loss: 0.5818
Epoch 13/20
3

In [19]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8') 
        input_ids = self.ids_from_chars(input_chars).to_tensor()
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature

        predicted_logits = predicted_logits

        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        predicted_chars = self.chars_from_ids(predicted_ids)
        return predicted_chars, states

one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


In [20]:
start = time.time()
states = None

next_char = tf.constant(['\n\n'])
result = [next_char]

for n in range(1000):
    next_char, states =  one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)
    
result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)




Lazen Cake
1 stick margarine
 cut in 1-inch pieces
 2 Tbsp. butter
 3 large ripe bananas
 skinned and boned
 1 can mushroom soup
 onion slices (2 c. red)k small pieces) or 1 lb. sharp Cheddar cheese
 2 cans chicken broth
 1 small onion
 chopped
 1 (13 oz.) can mandarin oranges
 sliced
 1/2 c. chopped pecans
 1/2 pkg. meale tyreal almonds are thin slice frozen onions
 salt
 pepper
 Parmesan cheesePlace pie plate potatoes.
 Increase heat to medium pieces. Heat milk, sugar and margarine in butch round stal water and dry to pist.
 Put shortening to brown top of cucumbers, walnuts.
 Heat between 20 minutes to the oven to moderate 30 minutes.
 Be sure to great balls in cookie sheet; serve warm with use side.
 Barter into pan.
 Next, melt almonds, put on cookie sheets.
 Set to ridge of fruit of your choice.
 Bake at 375\u00b0 for 10 minutes.

Special Corn Casserole
1 small head lettuce
 4 green onions
 sliced
 1/2 lb. grated cheese sour cream
 1 (10 oz.) pkg. frozen strawberries
 1 (12 oz.)